In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.1,
    )

In [14]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.schema.messages import SystemMessage
from langchain.agents.agent_toolkits import FileManagementToolkit
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper

def SearchToDDG(query):
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(query)

def SearchToWikipedia(query):
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

def write_file(input):
    file_path, content = input.split("|")
    file_path = file_path.strip()
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content.strip())

ddg_tool = Tool.from_function(
            func=SearchToDDG,
            name="Search To DuckDuckGo Function",
            description= "Use this function to search DuckDuckGo"
        )

wiki_tool = Tool.from_function(
            func=SearchToWikipedia,
            name="Search To Wikipedia Function",
            description="Use this function to search Wikipedia for factual information."
        )

write_tool = Tool.from_function(
            func=write_file,
            name="Save Research Function",
            description="Use this to save the research to a .txt file. Input format: filename.txt | content to write"
        )

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        ddg_tool,
        wiki_tool,
        write_tool,
    ],
    agent_kwargs={
        "system_message": SystemMessage(content="""
            You are a helpful assistant.
                                        
            You should try to search in Wikipedia or DuckDuckGo. If you find a website in
            DuckDuckGo, you should enter the website and extract it's content.
                                        
            You should finish by saving the research to a .txt file.
            """)
    }
)

agent.invoke("Research about XZ backdoor.")



> Entering new AgentExecutor chain...
Thought: I need to find information about the XZ backdoor.  A Wikipedia search might be good for a general overview, but I might need to supplement with a DuckDuckGo search for more up-to-date or less formally documented information.

Action: Search To Wikipedia Function
Action Input: XZ backdoor
Observation: Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely

{'input': 'Research about XZ backdoor.',
 'output': 'The XZ backdoor (CVE-2024-3094) was a critical vulnerability discovered in versions 5.6.0 and 5.6.1 of the XZ Utils compression library.  Introduced by an account using the name "Jia Tan" in February 2024, this backdoor allowed remote code execution via OpenSSH on affected Linux systems if an attacker possessed a specific Ed448 private key.  While not widely deployed to production systems at the time of discovery (March 29, 2024), it was present in development versions of major distributions.  The backdoor was sophisticated, using multi-stage obfuscation and encryption, and highlighted the importance of supply chain security, even in seemingly innocuous software like compression utilities.  The vulnerability was discovered by Andres Freund.  The CVSS score was 10.0, the highest possible.'}